# Install and Import packages

In [2]:
#!install_package_python37.sh add xlrd
#!install_package_python37.sh add nltk==3.7
#!install_package_python37.sh add python-dotenv
#!install_package_python37.sh add scikit-learn==0.21.1
#!install_package_python37.sh add pyjwt
#!install_package_python37.sh add boxsdk
#!install_package_python37.sh add openpyxl
#!install_package_python37.sh add sendgrid

#!install_package_python37.sh add gspread
#!install_package_python37.sh add oauth2client

In [2]:
# import necessary packages
import joblib
import os
import re
import string
from datetime import datetime, timezone,timedelta
import logging
from logging.handlers import TimedRotatingFileHandler
#from boxsdk import Client, JWTAuth
#import chardet
#from dotenv import load_dotenv
import io
import numpy as np
import pandas as pd
import json
import warnings
warnings.filterwarnings('ignore')
import nltk
import time
#nltk.download('popular')
#load_dotenv(".env")
#sg = sendgrid.SendGridAPIClient(os.environ.get('api_key'))


In [4]:
# Set up logger

LOGFILE = "logs/job3.log"

logger = logging.getLogger("dailylogs")

# set logging level : INFO, DEBUG, WARNING or ERROR
logger.setLevel(logging.DEBUG)


# Create TimedRotatingFileHandler with log file name
# It will create a new log file each day at midnight
handler = TimedRotatingFileHandler(LOGFILE, when="midnight", interval=1)

# This is the format in which logs will be displayed in log file
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# assign the formatter and suffix to file_handler object
# suffix will be added to each file
handler.setFormatter(formatter)
handler.suffix = "%Y%m%d"


# add the handler to logger
logger.addHandler(handler)

In [5]:
start_time = time.time()
logger.debug('New Job has started ....')

# Automated email notification

# cicd data import from box

In [8]:
data_cicd=pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/retrain_data/TaxML-CICD - Prod_Data_latest.csv',index_col=0)
#data_cicd=data_cicd.append(data_cicd_latest)
#logger.debug('Incremental CICD data has been added with shape {}'.format(data_cicd.shape))
#data_cicd=data_cicd.drop_duplicates()
#data_cicd.to_csv('./data/data_cicd_latest.csv',index=False)

In [9]:
data_cicd.shape

(583371, 15)

In [10]:
data_cicd.head()

,CICD Run Date,Date (BOT Sent Details to COE Team),UniqueUUID,store_uuid,item_uuid,Item,Description,establishment_type,CAT Name,Integer,Confidence Score,Agent Corrected CAT Name,Agent Corrected Integer,CAT NAME_ ValidationScore [0-100],Integer_ValidationScore[0-100]
11635,12/08/22 03:53,11/08/22 05:05,99c518e1-b510-5d2a-9ced-9bc7be9ab994:d7c97826-...,99c518e1-b510-5d2a-9ced-9bc7be9ab994,d7c97826-4684-4d85-beb0-18bcd9502c74,Gypsophila Peace Box,NaN,FLOWERS,CAT_TPP,531,0.27,CAT_TPP,531,100,100
11636,12/08/22 03:53,11/08/22 05:05,99c518e1-b510-5d2a-9ced-9bc7be9ab994:91c1d494-...,99c518e1-b510-5d2a-9ced-9bc7be9ab994,91c1d494-7690-4ccd-8dd7-334e499ff171,Chocolate Mousse,NaN,FLOWERS,CAT_PREPACKAGED_FOOD_CAKES,718,0.31,CAT_PREPACKAGED_FOOD_CAKES,718,100,100
11637,12/08/22 03:53,11/08/22 05:05,99c518e1-b510-5d2a-9ced-9bc7be9ab994:719b1381-...,99c518e1-b510-5d2a-9ced-9bc7be9ab994,719b1381-6b83-477a-bca4-0a3af84aaf8a,Chocolate Dream Box,NaN,FLOWERS,CAT_CONFECTIONARY,707,0.24,CAT_CONFECTIONARY,707,100,100
11638,12/08/22 03:53,11/08/22 05:05,99c518e1-b510-5d2a-9ced-9bc7be9ab994:22ef052d-...,99c518e1-b510-5d2a-9ced-9bc7be9ab994,22ef052d-6efd-47aa-b380-3e2af13fb9cd,Sweet Candy Basket,NaN,FLOWERS,CAT_CONFECTIONARY,707,0.56,CAT_CONFECTIONARY,707,100,100
11639,12/08/22 03:53,11/08/22 05:05,99c518e1-b510-5d2a-9ced-9bc7be9ab994:776f8287-...,99c518e1-b510-5d2a-9ced-9bc7be9ab994,776f8287-d635-4452-b6dd-4908e620a4f6,Hot Pink Gypsophila,NaN,FLOWERS,CAT_TPP,531,0.29,CAT_TPP,531,100,100


In [16]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

credentials = ServiceAccountCredentials.from_json_keyfile_name('./iacoe_sa.json', scope)
client = gspread.authorize(credentials)

spreadsheet = client.open('taxml-cicd-new')

with open('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/retrain_data/TaxML-CICD - Prod_Data_latest.csv', 'r') as file_obj:
    content = file_obj.read()
    client.import_csv(spreadsheet.id, data=content)

APIError: {'errors': [{'domain': 'global', 'reason': 'badRequest', 'message': 'Bad Request'}], 'code': 400, 'message': 'Bad Request'}

In [19]:
spreadsheet = client.open('taxml-cicd-new')
with open('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/retrain_data/mapping_latest.csv', 'r') as file_obj:
    content = file_obj.read()
    client.import_csv(spreadsheet.id, data=content)

In [22]:
spreadsheet.id

'1S6sZ0aOvyFUnoT6czyMO8E2MaI_3y0mn1ejfXCM-9yE'

In [23]:
file_obj

<_io.TextIOWrapper name='/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/retrain_data/mapping_latest.csv' mode='r' encoding='UTF-8'>